## Homework 07: Word Embeddings
<p> This homework is heavily based on <a href=https://github.com/oxford-cs-deepnlp-2017/practical-1>practical1</a> from
the <a href=https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/>Oxford CS - Deep NLP 2017</a> by Yannis Assael, Brendan Shillingford and Chris Dyer.

In [6]:
import numpy as np
import os
import re

from random import shuffle
print("Done")

Done


In [7]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()
print("Done")

Loading BokehJS ...

Done


### Part 0: Download the TED dataset

In [8]:
import urllib.request
import zipfile
import lxml.etree
print("Done")

Done


In [9]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.xml'):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/oxford-cs-deepnlp-2017/practical-1/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")
print("Done")

Done


In [11]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with open('ted_en-20160408.xml', 'rb') as f:
    doc = lxml.etree.parse(f)
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc
print("Done")

Done


### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [12]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]
print("Done")

Done


Let's start by removing all parenthesized strings using a regex:

In [13]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)
print("Done")

Done


We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [14]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]
print("Done")

Done


Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic.

In [15]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]
print("Done")

Done


Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [16]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)
print("Done")

Done


Two sample processed sentences:

In [17]:
print(sentences_ted[0])
print(sentences_ted[1])
print("Done")

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
Done


### Part 2: Word Frequencies

If you store (1) the counts of the top 1000 words in a list called `counts_ted_top1000` and (2) the actual words in `words_top_ted`, the code below will plot the histogram requested in the writeup.

In [18]:
from collections import defaultdict

# Count word frequencies in the TED corpus
counts_ted = defaultdict(int)

# sentences_ted should already exist earlier in the notebook as a list of tokenized sentences
for sent in sentences_ted:
    for w in sent:
        counts_ted[w] += 1

# Sort by frequency (descending)
sorted_ted = sorted(counts_ted.items(), key=lambda x: x[1], reverse=True)

# Take top 1000 (or fewer if vocab smaller)
top_k = min(1000, len(sorted_ted))
words_top_ted = [w for (w, c) in sorted_ted[:top_k]]
counts_ted_top1000 = [c for (w, c) in sorted_ted[:top_k]]

print(words_top_ted[:10])
print(counts_ted_top1000[:10])

print("Done")

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you']
[207748, 149305, 125169, 114818, 105399, 95146, 83180, 78070, 74738, 70923]
Done


In [26]:
import sys, subprocess

print("Using Python:", sys.executable)
subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib"])
print("✔ matplotlib installed")


Using Python: C:\Users\super\Documents\University of Arizona\UofA 2025 Fall\CSC 296\HW7\Enviroment\.env\Scripts\python.exe
✔ matplotlib installed


In [27]:
import matplotlib.pyplot as plt
print("✔ matplotlib import works")


✔ matplotlib import works


In [30]:
import matplotlib.pyplot as plt

# Top 20 TED words and counts
top20_words_ted = words_top_ted[:20]
top20_counts_ted = counts_ted_top1000[:20]

plt.figure(figsize=(10, 4))
plt.bar(top20_words_ted, top20_counts_ted)
plt.xticks(rotation=45, ha='right')
plt.title("Top 20 most frequent words (TED)")
plt.tight_layout()
plt.savefig("ted_top20_words.png", dpi=300)
plt.close()

print(list(zip(top20_words_ted, top20_counts_ted)))

print("Done")

[('the', 207748), ('and', 149305), ('to', 125169), ('of', 114818), ('a', 105399), ('that', 95146), ('i', 83180), ('in', 78070), ('it', 74738), ('you', 70923), ('we', 67629), ('is', 63251), ('s', 57156), ('this', 49241), ('so', 37014), ('they', 33102), ('was', 30806), ('for', 29713), ('are', 27995), ('have', 27344)]
Done


Plot distribution of top-1000 words

In [23]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100) #, normed=False)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)
print("Done")

Done


### Part 3: Train Word2Vec

In [31]:
# Full documentation: https://radimrehurek.com/gensim/models/word2vec.html
from gensim.models import Word2Vec
print("Done")

Done


In [32]:
model_ted = Word2Vec(sentences_ted, min_count=1)
print("Done")

Done


### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [33]:
print("\nTED – most similar to 'man':")
print(model_ted.wv.most_similar('man'))

print("\nTED – most similar to 'computer':")
print(model_ted.wv.most_similar('computer'))

print("\nTED – most similar to 'language':")
print(model_ted.wv.most_similar('language'))

print("\nTED – most similar to 'woman':")
print(model_ted.wv.most_similar('woman'))

print("\nTED – most similar to 'nice' + 'man':")
print(model_ted.wv.most_similar(positive=['nice', 'man']))

print("\nTED – most similar to 'nice' + 'woman':")
print(model_ted.wv.most_similar(positive=['nice', 'woman']))

print("Done")


TED – most similar to 'man':
[('woman', 0.8681902289390564), ('guy', 0.8151202201843262), ('lady', 0.7737560868263245), ('boy', 0.7554529905319214), ('girl', 0.7544465065002441), ('soldier', 0.7491644620895386), ('gentleman', 0.6998165845870972), ('kid', 0.6881040334701538), ('friend', 0.6873483061790466), ('king', 0.6802142858505249)]

TED – most similar to 'computer':
[('machine', 0.758583664894104), ('software', 0.7551513314247131), ('robot', 0.6989253163337708), ('printer', 0.6840587258338928), ('device', 0.6824505925178528), ('3d', 0.6753301024436951), ('program', 0.6748092770576477), ('interface', 0.6667406558990479), ('video', 0.6605533957481384), ('camera', 0.6536107063293457)]

TED – most similar to 'language':
[('culture', 0.6963860392570496), ('mathematics', 0.6529236435890198), ('desire', 0.6520811915397644), ('logic', 0.6434112787246704), ('english', 0.641686737537384), ('nature', 0.6308330297470093), ('interaction', 0.6307618021965027), ('narrative', 0.6299844980239868),

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [49]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted.wv[words_top_ted]
print("Done")

Done


In [50]:
from sklearn.manifold import TSNE
import numpy as np

# Use up to 1000 most frequent words
words_for_tsne_ted = words_top_ted[:1000]
vecs_ted = np.array([model_ted.wv[w] for w in words_for_tsne_ted if w in model_ted.wv])

tsne_ted = TSNE(n_components=2, init="random", random_state=0, perplexity=30)
embed_ted = tsne_ted.fit_transform(vecs_ted)

plt.figure(figsize=(12, 10))
plt.scatter(embed_ted[:, 0], embed_ted[:, 1], s=3)
plt.title("t-SNE of TED Embeddings")
plt.tight_layout()
plt.savefig("tsne_ted.png", dpi=300)
plt.close()

print("Done")

Done


In [51]:
months = ["january", "february", "march", "april", "may", "june",
          "july", "august", "september", "october", "november", "december"]

time_units = ["seconds", "minutes", "hours", "days", "weeks", "months", "years"]

words_time_ted = [w for w in months + time_units if w in model_ted.wv]

print("TED – words available in model:")
print(words_time_ted)

vecs_time_ted = np.array([model_ted.wv[w] for w in words_time_ted])

tsne_time_ted = TSNE(n_components=2, init="random", random_state=0, perplexity=10)
embed_time_ted = tsne_time_ted.fit_transform(vecs_time_ted)

plt.figure(figsize=(8, 6))
plt.scatter(embed_time_ted[:, 0], embed_time_ted[:, 1], s=20)

for i, w in enumerate(words_time_ted):
    plt.text(embed_time_ted[i, 0] + 0.3,
             embed_time_ted[i, 1] + 0.3,
             w, fontsize=9)

plt.title("TED – Months vs Time Units")
plt.tight_layout()
plt.savefig("ted_months_vs_time_units.png", dpi=300)
plt.close()

print("Done")

TED – words available in model:
['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years']
Done


In [52]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)
print("Done")

Done


In [53]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)
print("Done")

Done


### Part 5: Wiki Learnt Representations

Download dataset

In [34]:
# HL: Load Wikitext edited for CSC 296; previous code raises 301 error
from datasets import load_dataset

dataset = load_dataset('iohadrubin/wikitext-103-raw-v1')
raw_data = dataset["train"]["text"]
input_text = "\n".join(raw_data)
print("Done")

dataset_infos.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

C:\Users\super\Documents\University of Arizona\UofA 2025 Fall\CSC 296\HW7\Enviroment\.env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\super\.cache\huggingface\hub\datasets--iohadrubin--wikitext-103-raw-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00002-b755d19de94348(…):   0%|          | 0.00/148M [00:00<?, ?B/s]

data/train-00001-of-00002-0bf6d0c487c2e7(…):   0%|          | 0.00/152M [00:00<?, ?B/s]

data/validation-00000-of-00001-4c0139624(…):   0%|          | 0.00/631k [00:00<?, ?B/s]

data/test-00000-of-00001-b7859cf6365689a(…):   0%|          | 0.00/707k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29567 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/62 [00:00<?, ? examples/s]

Done


Preprocess sentences (note that it's important to remove small sentences for performance)

In [35]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
print("Done")

Done


In [36]:
# sample 1/5 of the data
shuffle(sentences_wiki)
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print("Done")

Done


In [37]:
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split()
print("Done")

Done


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [38]:
model_wiki = Word2Vec(sentences_wiki, min_count=1)
print("Done")

Done


In [39]:
print("\nWIKI – most similar to 'man':")
print(model_wiki.wv.most_similar('man'))

print("\nWIKI – most similar to 'computer':")
print(model_wiki.wv.most_similar('computer'))

print("\nWIKI – most similar to 'language':")
print(model_wiki.wv.most_similar('language'))

print("\nWIKI – most similar to 'woman':")
print(model_wiki.wv.most_similar('woman'))

print("\nWIKI – most similar to 'nice' + 'man':")
print(model_wiki.wv.most_similar(positive=['nice', 'man']))

print("\nWIKI – most similar to 'nice' + 'woman':")
print(model_wiki.wv.most_similar(positive=['nice', 'woman']))

print("Done")


WIKI – most similar to 'man':
[('woman', 0.7414947152137756), ('boy', 0.6574636697769165), ('girl', 0.6221573948860168), ('person', 0.6065489053726196), ('soldier', 0.5937392115592957), ('joke', 0.5922864079475403), ('creature', 0.5738527178764343), ('mask', 0.5601330995559692), ('lad', 0.5554105043411255), ('men', 0.5524371862411499)]

WIKI – most similar to 'computer':
[('software', 0.8009969592094421), ('user', 0.775404155254364), ('interface', 0.7740408778190613), ('hardware', 0.7630912065505981), ('computers', 0.7612613439559937), ('technology', 0.7423678636550903), ('valve', 0.716158390045166), ('simulation', 0.7152251601219177), ('cgi', 0.7100728154182434), ('controller', 0.7074329257011414)]

WIKI – most similar to 'language':
[('literature', 0.7573751211166382), ('languages', 0.7562223076820374), ('dialect', 0.7187418341636658), ('vocabulary', 0.7180837392807007), ('translation', 0.6968158483505249), ('folklore', 0.6875981092453003), ('alphabet', 0.6802295446395874), ('cultur

In [40]:
import collections

# Wiki frequencies
counts_wiki = defaultdict(int)
for sent in sentences_wiki:
    for w in sent:
        counts_wiki[w] += 1

sorted_wiki = sorted(counts_wiki.items(), key=lambda x: x[1], reverse=True)

top_k_wiki = min(1000, len(sorted_wiki))
words_top_wiki = [w for (w, c) in sorted_wiki[:top_k_wiki]]
counts_wiki_top1000 = [c for (w, c) in sorted_wiki[:top_k_wiki]]

print(words_top_wiki[:10])
print(counts_wiki_top1000[:10])

print("Done")

['the', 'of', 'and', 'in', 'to', 'a', 'was', 's', 'on', 'as']
[1284924, 547190, 499506, 432600, 398284, 357284, 214426, 162879, 153438, 144282]
Done


In [41]:
import matplotlib.pyplot as plt

top20_words_wiki = words_top_wiki[:20]
top20_counts_wiki = counts_wiki_top1000[:20]

plt.figure(figsize=(10, 4))
plt.bar(top20_words_wiki, top20_counts_wiki)
plt.xticks(rotation=45, ha='right')
plt.title("Top 20 Most Frequent Words (Wiki)")
plt.tight_layout()
plt.savefig("wiki_top20_words.png", dpi=300)
plt.close()

print(list(zip(top20_words_wiki, top20_counts_wiki)))

print("Done")

[('the', 1284924), ('of', 547190), ('and', 499506), ('in', 432600), ('to', 398284), ('a', 357284), ('was', 214426), ('s', 162879), ('on', 153438), ('as', 144282), ('for', 139753), ('that', 137935), ('with', 129671), ('by', 125626), ('is', 102606), ('his', 92989), ('at', 91718), ('he', 91067), ('from', 88621), ('it', 88248)]
Done


#### t-SNE visualization

In [44]:
import sys, subprocess

print("Using Python:", sys.executable)
subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
print("✔ scikit-learn installed successfully")


Using Python: C:\Users\super\Documents\University of Arizona\UofA 2025 Fall\CSC 296\HW7\Enviroment\.env\Scripts\python.exe
✔ scikit-learn installed successfully


In [45]:
from sklearn.manifold import TSNE
print("✔ sklearn import works")


✔ sklearn import works


In [46]:
from sklearn.manifold import TSNE
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki.wv[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)
print("Done")

Done


In [47]:
words_for_tsne_wiki = words_top_wiki[:1000]
vecs_wiki = np.array([model_wiki.wv[w] for w in words_for_tsne_wiki if w in model_wiki.wv])

tsne_wiki = TSNE(n_components=2, init="random", random_state=0, perplexity=30)
embed_wiki = tsne_wiki.fit_transform(vecs_wiki)

plt.figure(figsize=(12, 10))
plt.scatter(embed_wiki[:, 0], embed_wiki[:, 1], s=3)
plt.title("t-SNE of Wiki Embeddings")
plt.tight_layout()
plt.savefig("tsne_wiki.png", dpi=300)
plt.close()

print("Done")

Done


In [54]:
words_time_wiki = [w for w in months + time_units if w in model_wiki.wv]

print("Wiki – words available in model:")
print(words_time_wiki)

vecs_time_wiki = np.array([model_wiki.wv[w] for w in words_time_wiki])

tsne_time_wiki = TSNE(n_components=2, init="random", random_state=0, perplexity=10)
embed_time_wiki = tsne_time_wiki.fit_transform(vecs_time_wiki)

plt.figure(figsize=(8, 6))
plt.scatter(embed_time_wiki[:, 0], embed_time_wiki[:, 1], s=20)

for i, w in enumerate(words_time_wiki):
    plt.text(embed_time_wiki[i, 0] + 0.3,
             embed_time_wiki[i, 1] + 0.3,
             w, fontsize=9)

plt.title("Wiki – Months vs Time Units")
plt.tight_layout()
plt.savefig("wiki_months_vs_time_units.png", dpi=300)
plt.close()

print("Done")

Wiki – words available in model:
['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years']
Done


In [55]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)
print("Done")

Done


In [57]:
import matplotlib.pyplot as plt

def save_similarity_plot(words, scores, title, filename):
    plt.figure(figsize=(10,4))
    plt.bar(words, scores)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(min(scores)-0.05, max(scores)+0.05)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print("✔ Saved:", filename)
# TED data
ted_man_words = ["woman", "guy", "lady", "boy", "girl"]
ted_man_scores = [0.8682, 0.8151, 0.7738, 0.7555, 0.7544]

ted_woman_words = ["man", "girl", "boy", "lady", "child"]
ted_woman_scores = [0.8682, 0.8590, 0.8056, 0.7962, 0.7673]

ted_computer_words = ["machine", "software", "robot", "printer", "device"]
ted_computer_scores = [0.7586, 0.7552, 0.6989, 0.6841, 0.6825]

ted_language_words = ["culture", "mathematics", "desire", "logic", "english"]
ted_language_scores = [0.6964, 0.6529, 0.6521, 0.6434, 0.6417]

ted_nice_man_words = ["guy", "boy", "girl", "woman", "kid"]
ted_nice_man_scores = [0.7432, 0.7364, 0.7126, 0.7088, 0.6954]

ted_nice_woman_words = ["girl", "boy", "kid", "lady", "mary"]
ted_nice_woman_scores = [0.7632, 0.7514, 0.7162, 0.6918, 0.6897]
# WIKI data
wiki_man_words = ["woman", "boy", "girl", "person", "soldier"]
wiki_man_scores = [0.7415, 0.6575, 0.6222, 0.6065, 0.5937]

wiki_woman_words = ["girl", "child", "man", "herself", "person"]
wiki_woman_scores = [0.7955, 0.7475, 0.7415, 0.6708, 0.6669]

wiki_computer_words = ["software", "user", "interface", "hardware", "computers"]
wiki_computer_scores = [0.8010, 0.7754, 0.7740, 0.7631, 0.7613]

wiki_language_words = ["literature", "languages", "dialect", "vocabulary", "translation"]
wiki_language_scores = [0.7574, 0.7562, 0.7187, 0.7181, 0.6968]

wiki_nice_man_words = ["really", "fun", "boy", "bit", "girl"]
wiki_nice_man_scores = [0.6810, 0.6729, 0.6714, 0.6701, 0.6688]

wiki_nice_woman_words = ["girl", "baby", "smile", "sad", "sexy"]
wiki_nice_woman_scores = [0.7637, 0.6833, 0.6825, 0.6741, 0.6683]
save_similarity_plot(ted_man_words, ted_man_scores,
                     "TED – Similar Words to man", "ted_man_similar.png")

save_similarity_plot(ted_woman_words, ted_woman_scores,
                     "TED – Similar Words to woman", "ted_woman_similar.png")

save_similarity_plot(ted_computer_words, ted_computer_scores,
                     "TED – Similar Words to computer", "ted_computer_similar.png")

save_similarity_plot(ted_language_words, ted_language_scores,
                     "TED – Similar Words to language", "ted_language_similar.png")

save_similarity_plot(ted_nice_man_words, ted_nice_man_scores,
                     "TED – Similar Words to nice + man", "ted_niceman_similar.png")

save_similarity_plot(ted_nice_woman_words, ted_nice_woman_scores,
                     "TED – Similar Words to nice + woman", "ted_nicewoman_similar.png")
save_similarity_plot(wiki_man_words, wiki_man_scores,
                     "WIKI – Similar Words to man", "wiki_man_similar.png")

save_similarity_plot(wiki_woman_words, wiki_woman_scores,
                     "WIKI – Similar Words to woman", "wiki_woman_similar.png")

save_similarity_plot(wiki_computer_words, wiki_computer_scores,
                     "WIKI – Similar Words to computer", "wiki_computer_similar.png")

save_similarity_plot(wiki_language_words, wiki_language_scores,
                     "WIKI – Similar Words to language", "wiki_language_similar.png")

save_similarity_plot(wiki_nice_man_words, wiki_nice_man_scores,
                     "WIKI – Similar Words to nice + man", "wiki_niceman_similar.png")

save_similarity_plot(wiki_nice_woman_words, wiki_nice_woman_scores,
                     "WIKI – Similar Words to nice + woman", "wiki_nicewoman_similar.png")


✔ Saved: ted_man_similar.png
✔ Saved: ted_woman_similar.png
✔ Saved: ted_computer_similar.png
✔ Saved: ted_language_similar.png
✔ Saved: ted_niceman_similar.png
✔ Saved: ted_nicewoman_similar.png
✔ Saved: wiki_man_similar.png
✔ Saved: wiki_woman_similar.png
✔ Saved: wiki_computer_similar.png
✔ Saved: wiki_language_similar.png
✔ Saved: wiki_niceman_similar.png
✔ Saved: wiki_nicewoman_similar.png
